In [1]:
from ase import Atoms
from ase.io import read, write
import xcquinox as xce
import torch, jax, optax
import numpy as np
import equinox as eqx
import jax.numpy as jnp
import pyscfad as psa
import os, sys, pickle
from pyscf import dft, scf, gto, df
from pyscf.pbc import scf as scfp
from pyscf.pbc import gto as gtop
from pyscf.pbc import dft as dftp
# from pyscfad.pbc import scf as scfp
# from pyscfad.pbc import gto as gtop
# from pyscfad.pbc import dft as dftp

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"]="platform"
import pandas as pd
from mp_api.client import MPRester
from mldftdat.density import get_exchange_descriptors2
from mldftdat.analyzers import RKSAnalyzer
from emmet.core.summary import HasProps
from ase.build import bulk
from pymatgen.io.ase import AseAtomsAdaptor

os.environ['JAX_LOG_COMPILES'] = '1'
jax.config.update('jax_log_compiles', True)

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
from xcquinox.net import eX, eC

In [3]:
refdp = '/home/awills/Documents/Research/datasets/borlido2019/smalls_data'
trjs = sorted([i for i in os.listdir(refdp) if '_' in i and 'traj' in i and 'smalls' not in i], key = lambda x: int(x.split('_')[0]))
mols = []
mfs = []
rdms = []
init_dms = []
rmoes = []
rmooccs = []
rmocoeffs = []
refgaps = []
reftotes = []
gws = []
gcs = []
aos = []
ts = []
hcs = []
eris = []
ss = []
ogds = []
GRID_SUBSAMPLE_SIZE=100
for idx, i in enumerate(trjs):
    atoms = read(os.path.join(refdp, i))
    fbase = i.split('.')[0]
    rdm = np.load(os.path.join(refdp, fbase+'.dm.npy'))
    if rdm.shape[0] < 19:
        rdms.append(rdm)
        ogds.append(rdm.shape)
        refgaps.append(atoms.info['reference'])
        rmoes.append(np.load(os.path.join(refdp, fbase+'.mo_energy.npy')))
        rmooccs.append(np.load(os.path.join(refdp, fbase+'.mo_occ.npy')))
        rmocoeffs.append(np.load(os.path.join(refdp, fbase+'.mo_coeff.npy')))
        print(atoms, atoms.info, atoms.calc.results)
        atoms.info['e_calc'] = atoms.calc.results['energy']
        reftotes.append(atoms.info['e_calc'])
        pos = atoms.positions
        spec = atoms.get_chemical_symbols()
        mol_input = [[s,p] for s,p in zip(spec,pos)]
        cell = np.array(atoms.cell)
        mol = gtop.Cell(a=cell, rcut=0.1, atom=mol_input, basis='lanl2dz', charge=0, pseudo='gth-pbe', verbose=9, spin=0)
        mol.exp_to_discard = 0.1
        mol.build()
        mols.append(mol)
        mf = dftp.RKS(mol)
        mf.grids.level = 1
        mf.grids.build()
        gw = mf.grids.weights[::GRID_SUBSAMPLE_SIZE]
        gc = mf.grids.coords[::GRID_SUBSAMPLE_SIZE]
        gws.append(gw)
        gcs.append(gc)
        # aos.append(mf._numint.eval_ao(mol, mf.grids.coords, deriv=2))
        init_dms.append(mf.get_init_guess())
        aos.append(jnp.array(mf._numint.eval_ao(mol, gc, deriv=2)))
        hcs.append(jnp.array(mf.get_hcore()))
        ts.append(jnp.array(mol.intor('int1e_kin')))
        eris.append(jnp.array(mol.intor('int2e')))
        # print('eri shape', eri.shape)
        # print('ao_eval shape', ao_eval.shape)
        ss.append(jnp.linalg.inv(jnp.linalg.cholesky(mol.intor('int1e_ovlp'))))
        mfs.append(mf)

Atoms(symbols='Ar', pbc=True, cell=[[0.0, 2.820386, 2.820386], [2.820386, 0.0, 2.820386], [2.820386, 2.820386, 0.0]], calculator=SinglePointCalculator(...)) {'borlido_idx': 35, 'mp_id': 'mp-23155', 'reference': 14.15} {'energy': -19.14554484483416}
arg.atm = [[ 8 20  1 23  0  0]]
arg.bas = [[ 0  0  2  1  0 24 26  0]
 [ 0  0  1  1  0 28 29  0]
 [ 0  1  2  1  0 30 32  0]
 [ 0  1  1  1  0 34 35  0]]
arg.env = [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  2.613       0.5736     -2.65356551  2.11503642  0.2014      0.75955569
  7.86        0.7387     -2.13150432  2.02105235  0.2081      0.41003836]
ecpbas  = []
System: uname_result(system='Linux', node='aegis', release='5.15.0-101-generic', version='#111~20.04.1-Ubuntu SMP Mon Mar 11 15:44:43 UTC 2024', machine='x86_64')  Thread

Finished tracing + transforming convert_element_type for pjit in 0.00033354759216308594 sec
Compiling convert_element_type for with global shapes and types [ShapedArray(float64[10,8574,8])]. Argument mapping: (GSPMDSharding({replicated}),).
Finished jaxpr to MLIR module conversion jit(convert_element_type) in 0.0013041496276855469 sec
2024-04-11 18:08:02.120224: W external/xla/xla/service/gpu/nvptx_compiler.cc:742] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.4.99). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
Finished XLA compilation of jit(convert_element_type) in 0.7219085693359375 sec
Finished tracing + transforming convert_element_type for pjit in 0.0002448558807373047 sec
Compiling convert_element_type for with global shapes and types [ShapedArray(float64[8,8])

Atoms(symbols='KBr', pbc=True, cell=[[-3.351538, -3.351538, 0.0], [-3.351538, 0.0, -3.351538], [0.0, -3.351538, -3.351538]], calculator=SinglePointCalculator(...)) {'borlido_idx': 262, 'mp_id': 'mp-23251', 'reference': 7.8} {'energy': -38.416505032573}
arg.atm = [[ 9 20  1 23  0  0]
 [ 7 24  1 27  0  0]]
arg.bas = [[ 0  0  4  2  0 40 44  0]
 [ 0  0  1  1  0 52 53  0]
 [ 0  1  3  1  0 54 57  0]
 [ 0  1  1  1  0 60 61  0]
 [ 0  1  1  1  0 62 63  0]
 [ 1  0  2  1  0 28 30  0]
 [ 1  0  1  1  0 32 33  0]
 [ 1  1  2  1  0 34 36  0]
 [ 1  1  1  1  0 38 39  0]]
arg.env = [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -6.33348892 -6.33348892 -6.33348892  0.          1.159       0.7107
 -8.57329507  6.59109833  0.1905      0.72851124  2.691       0.4446
 -1.19632857  1.10407922  0.1377

Finished tracing + transforming convert_element_type for pjit in 0.00025844573974609375 sec
Compiling convert_element_type for with global shapes and types [ShapedArray(float64[10,14429,13])]. Argument mapping: (GSPMDSharding({replicated}),).
Finished jaxpr to MLIR module conversion jit(convert_element_type) in 0.0009741783142089844 sec
Finished XLA compilation of jit(convert_element_type) in 0.0024819374084472656 sec
Finished tracing + transforming convert_element_type for pjit in 0.0002474784851074219 sec
Compiling convert_element_type for with global shapes and types [ShapedArray(float64[13,13])]. Argument mapping: (GSPMDSharding({replicated}),).
Finished jaxpr to MLIR module conversion jit(convert_element_type) in 0.0011446475982666016 sec
Finished XLA compilation of jit(convert_element_type) in 0.002882242202758789 sec
Finished tracing + transforming convert_element_type for pjit in 0.0002543926239013672 sec
Compiling convert_element_type for with global shapes and types [ShapedAr

Atoms(symbols='KCl', pbc=True, cell=[[0.0, 3.197645, 3.197645], [3.197645, 0.0, 3.197645], [3.197645, 3.197645, 0.0]], calculator=SinglePointCalculator(...)) {'borlido_idx': 263, 'mp_id': 'mp-23193', 'reference': 8.69} {'energy': -39.87072403716726}
arg.atm = [[ 9 20  1 23  0  0]
 [ 7 24  1 27  0  0]]
arg.bas = [[ 0  0  4  2  0 40 44  0]
 [ 0  0  1  1  0 52 53  0]
 [ 0  1  3  1  0 54 57  0]
 [ 0  1  1  1  0 60 61  0]
 [ 0  1  1  1  0 62 63  0]
 [ 1  0  2  1  0 28 30  0]
 [ 1  0  1  1  0 32 33  0]
 [ 1  1  2  1  0 34 36  0]
 [ 1  1  1  1  0 38 39  0]]
arg.env = [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  6.04267329  6.04267329  6.04267329  0.          2.231       0.472
 -2.26015379  1.8045213   0.1631      0.64841865  6.296       0.6333
 -1.84938436  1.67144551  0.1819    

...

Finished tracing + transforming convert_element_type for pjit in 0.0002620220184326172 sec
Compiling convert_element_type for with global shapes and types [ShapedArray(float64[10,10304,16])]. Argument mapping: (GSPMDSharding({replicated}),).
Finished jaxpr to MLIR module conversion jit(convert_element_type) in 0.0016083717346191406 sec
Finished XLA compilation of jit(convert_element_type) in 0.002927064895629883 sec


Atoms(symbols='RbI', pbc=True, cell=[[0.0, 3.743926, 3.743926], [3.743926, 0.0, 3.743926], [3.743926, 3.743926, 0.0]], calculator=SinglePointCalculator(...)) {'borlido_idx': 355, 'mp_id': 'mp-22903', 'reference': 6.37} {'energy': -32.515360723015746}
arg.atm = [[ 9 20  1 23  0  0]
 [ 7 24  1 27  0  0]]
arg.bas = [[ 0  0  4  2  0 40 44  0]
 [ 0  0  1  1  0 52 53  0]
 [ 0  1  3  1  0 54 57  0]
 [ 0  1  2  1  0 60 62  0]
 [ 0  1  1  1  0 64 65  0]
 [ 1  0  2  1  0 28 30  0]
 [ 1  0  1  1  0 32 33  0]
 [ 1  1  2  1  0 34 36  0]
 [ 1  1  1  1  0 38 39  0]]
arg.env = [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  7.07499477  7.07499477  7.07499477  0.          0.7242      0.4653
 -5.89683703  4.95722796  0.1336      0.55830288  1.29        0.318
 -0.8391923   0.76878395  0.1053   

Finished tracing + transforming convert_element_type for pjit in 0.00025200843811035156 sec
Compiling convert_element_type for with global shapes and types [ShapedArray(float64[10,5315,16])]. Argument mapping: (GSPMDSharding({replicated}),).
Finished jaxpr to MLIR module conversion jit(convert_element_type) in 0.001384735107421875 sec
Finished XLA compilation of jit(convert_element_type) in 0.0028107166290283203 sec


Atoms(symbols='Xe', pbc=True, cell=[[0.0, 3.330738, 3.330738], [3.330738, 0.0, 3.330738], [3.330738, 3.330738, 0.0]], calculator=SinglePointCalculator(...)) {'borlido_idx': 446, 'mp_id': 'mp-611517', 'reference': 9.29} {'energy': -13.863584578602147}
arg.atm = [[ 8 20  1 23  0  0]]
arg.bas = [[ 0  0  2  1  0 24 26  0]
 [ 0  0  1  1  0 28 29  0]
 [ 0  1  2  1  0 30 32  0]
 [ 0  1  1  1  0 34 35  0]]
arg.env = [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.7646      0.5322     -7.54913164  6.56063616  0.1491      0.60621022
  1.211       0.3808     -0.96985254  1.00972306  0.1259      0.2187845 ]
ecpbas  = []
System: uname_result(system='Linux', node='aegis', release='5.15.0-101-generic', version='#111~20.04.1-Ubuntu SMP Mon Mar 11 15:44:43 UTC 2024', machine='x86_64')  Thre

Finished tracing + transforming convert_element_type for pjit in 0.0002684593200683594 sec
Compiling convert_element_type for with global shapes and types [ShapedArray(float64[10,1039,8])]. Argument mapping: (GSPMDSharding({replicated}),).
Finished jaxpr to MLIR module conversion jit(convert_element_type) in 0.0011262893676757812 sec
Finished XLA compilation of jit(convert_element_type) in 0.002789735794067383 sec


In [4]:
ogds

[(8, 8), (13, 13), (13, 13), (8, 8), (16, 16), (16, 16), (16, 16), (8, 8)]

In [5]:
rdms = xce.utils.pad_array_list(rdms)
init_dms = xce.utils.pad_array_list(init_dms)
rmoes = xce.utils.pad_array_list(rmoes)
rmooccs = xce.utils.pad_array_list(rmooccs)
rmocoeffs = xce.utils.pad_array_list(rmocoeffs)
gws = xce.utils.pad_array_list(gws)
gcs = xce.utils.pad_array_list(gcs)
aos = xce.utils.pad_array_list(aos)
hcs = xce.utils.pad_array_list(hcs)
ts = xce.utils.pad_array_list(ts)
eris = xce.utils.pad_array_list(eris)
ss = xce.utils.pad_array_list(ss)

Finished tracing + transforming _pad for pjit in 0.000896453857421875 sec
Compiling _pad for with global shapes and types [ShapedArray(float64[8,8]), ShapedArray(int64[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
Finished jaxpr to MLIR module conversion jit(_pad) in 0.0018398761749267578 sec
Finished XLA compilation of jit(_pad) in 0.00908041000366211 sec
Finished tracing + transforming _pad for pjit in 0.0010142326354980469 sec
Compiling _pad for with global shapes and types [ShapedArray(float64[13,13]), ShapedArray(int64[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
Finished jaxpr to MLIR module conversion jit(_pad) in 0.0018298625946044922 sec
Finished XLA compilation of jit(_pad) in 0.009118080139160156 sec
Finished tracing + transforming _pad for pjit in 0.0008685588836669922 sec
Compiling _pad for with global shapes and types [ShapedArray(float64[16,16]), ShapedArray(int64[]

[16, 16] (8, 8)
[16, 16] (13, 13)
[16, 16] (13, 13)
[16, 16] (8, 8)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (8, 8)
[16, 16] (8, 8)
[16, 16] (13, 13)
[16, 16] (13, 13)
[16, 16] (8, 8)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (8, 8)
[16] (8,)
[16] (13,)
[16] (13,)
[16] (8,)
[16] (16,)
[16] (16,)
[16] (16,)
[16] (8,)
[16] (8,)
[16] (13,)
[16] (13,)
[16] (8,)
[16] (16,)
[16] (16,)
[16] (16,)
[16] (8,)
[16, 16] (8, 8)
[16, 16] (13, 13)
[16, 16] (13, 13)
[16, 16] (8, 8)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (8, 8)
[14429] (8574,)
[14429] (14429,)
[14429] (12951,)
[14429] (3286,)
[14429] (4566,)
[14429] (10304,)
[14429] (5315,)
[14429] (1039,)
[14429, 3] (8574, 3)


Finished XLA compilation of jit(_pad) in 0.008038997650146484 sec
Finished tracing + transforming _pad for pjit in 0.0010881423950195312 sec
Compiling _pad for with global shapes and types [ShapedArray(float64[14429,3]), ShapedArray(int64[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
Finished jaxpr to MLIR module conversion jit(_pad) in 0.0018393993377685547 sec
Finished XLA compilation of jit(_pad) in 0.00580143928527832 sec
Finished tracing + transforming _pad for pjit in 0.000885009765625 sec
Compiling _pad for with global shapes and types [ShapedArray(float64[12951,3]), ShapedArray(int64[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
Finished jaxpr to MLIR module conversion jit(_pad) in 0.001970052719116211 sec
Finished XLA compilation of jit(_pad) in 0.008351325988769531 sec
Finished tracing + transforming _pad for pjit in 0.0008807182312011719 sec
Compiling _pad for with globa

[14429, 3] (14429, 3)
[14429, 3] (12951, 3)
[14429, 3] (3286, 3)
[14429, 3] (4566, 3)
[14429, 3] (10304, 3)
[14429, 3] (5315, 3)
[14429, 3] (1039, 3)
[10, 14429, 16] (10, 8574, 8)
[10, 14429, 16] (10, 14429, 13)
[10, 14429, 16] (10, 12951, 13)
[10, 14429, 16] (10, 3286, 8)
[10, 14429, 16] (10, 4566, 16)


Finished XLA compilation of jit(_pad) in 0.01147150993347168 sec
Finished tracing + transforming _pad for pjit in 0.0012950897216796875 sec
Compiling _pad for with global shapes and types [ShapedArray(float64[10,10304,16]), ShapedArray(int64[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
Finished jaxpr to MLIR module conversion jit(_pad) in 0.002027273178100586 sec
Finished XLA compilation of jit(_pad) in 0.011333703994750977 sec
Finished tracing + transforming _pad for pjit in 0.0011086463928222656 sec
Compiling _pad for with global shapes and types [ShapedArray(float64[10,5315,16]), ShapedArray(int64[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
Finished jaxpr to MLIR module conversion jit(_pad) in 0.002045154571533203 sec
Finished XLA compilation of jit(_pad) in 0.01152658462524414 sec
Finished tracing + transforming _pad for pjit in 0.0010638236999511719 sec
Compiling _pad for w

[10, 14429, 16] (10, 10304, 16)
[10, 14429, 16] (10, 5315, 16)
[10, 14429, 16] (10, 1039, 8)
[16, 16] (8, 8)
[16, 16] (13, 13)
[16, 16] (13, 13)
[16, 16] (8, 8)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (8, 8)
[16, 16] (8, 8)
[16, 16] (13, 13)
[16, 16] (13, 13)
[16, 16] (8, 8)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (8, 8)
[16, 16, 16, 16] (8, 8, 8, 8)
[16, 16, 16, 16] (13, 13, 13, 13)
[16, 16, 16, 16] (13, 13, 13, 13)
[16, 16, 16, 16] (8, 8, 8, 8)
[16, 16, 16, 16] (16, 16, 16, 16)
[16, 16, 16, 16] (16, 16, 16, 16)
[16, 16, 16, 16] (16, 16, 16, 16)
[16, 16, 16, 16] (8, 8, 8, 8)
[16, 16] (8, 8)
[16, 16] (13, 13)
[16, 16] (13, 13)
[16, 16] (8, 8)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (16, 16)
[16, 16] (8, 8)


In [6]:
rmoes[0].shape

(16,)

In [7]:
class E_Rho_Gap_loss(eqx.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, model, mol, mf, gc, gw, inp_dm, mo_occ, ogd, refE, refDM, refGap):

        #get static matrices from mol and mf
        ao_eval = jnp.array(mf._numint.eval_ao(mol, gc, deriv=2))
        t = jnp.array(mol.intor('int1e_kin'))
        vn = jnp.array(mol.intor('int1e_nuc'))
        hc = jnp.array(mf.get_hcore())
        eri = jnp.array(mol.intor('int2e'))
        s = jnp.linalg.inv(jnp.linalg.cholesky(mol.intor('int1e_ovlp')))
        homo_i = jnp.max(jnp.nonzero(mo_occ, size=inp_dm.shape[0])[0])


        #vxc function for gradient
        vgf = lambda x: model(x, ao_eval, mf.grids.weights)
        print('Generating dmp, moep, mocoep')
        dmp, moep, mocoep = xce.utils.get_dm_moe(inp_dm, eri, vgf, mo_occ, hc, s, ogd)

        veff = xce.utils.get_veff()(dmp, eri)
        print('Getting exc, vxc from dmp')
        exc, vxc = eqx.filter_value_and_grad(model)(dmp, ao_eval, gw)
        e_pred = xce.utils.energy_tot()(dmp, hc, veff+vxc+vn)
        # rho_pred = xce.utils.get_rho()(dmp, ao_eval)
        # rho_ref = xce.utils.get_rho()(refDM, ao_eval)
        print('Getting rho_pred and rho_ref')
        rho_pred = jnp.einsum('xij,yik,...jk->xy...i', ao_eval, ao_eval, dmp)[0,0]+1e-10
        rho_ref = jnp.einsum('xij,yik,...jk->xy...i', ao_eval, ao_eval, refDM)[0,0]+1e-10
        print('Getting gap_pred')
        gap_pred = moep[homo_i+1]-moep[homo_i]

        eL = (e_pred-refE)**2

        rho_L = jnp.sum(gw*(rho_pred-rho_ref)**2)/mol.nelectron**2

        gap_L = (gap_pred-refGap)**2

        return jnp.sqrt(eL+rho_L+gap_L)[0]


class E_DM_Gap_loss(eqx.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, model, mol, mf, gc, gw, inp_dm, mo_occ, ogd, refE, refDM, refGap):

        #get static matrices from mol and mf
        ao_eval = jnp.array(jax.lax.stop_gradient(mf._numint.eval_ao(mol, gc, deriv=2)))
        t = jnp.array(mol.intor('int1e_kin'))
        vn = jnp.array(mol.intor('int1e_nuc'))
        hc = jnp.array(mf.get_hcore())
        eri = jnp.array(mol.intor('int2e'))
        # print('eri shape', eri.shape)
        # print('ao_eval shape', ao_eval.shape)
        s = jnp.linalg.inv(jnp.linalg.cholesky(mol.intor('int1e_ovlp')))
        homo_i = jnp.max(jnp.nonzero(mo_occ, size=inp_dm.shape[0])[0])

        ao_eval = xce.utils.pad_array(ao_eval, ao_eval, shape=(ao_eval.shape[0], gw.shape[0], inp_dm.shape[0]))
        s = xce.utils.pad_array(s, inp_dm)
        t = xce.utils.pad_array(t, inp_dm)
        hc = xce.utils.pad_array(hc, inp_dm)
        vn = xce.utils.pad_array(vn, inp_dm)
        eri = xce.utils.pad_array(eri, eri, shape=(inp_dm.shape[0], inp_dm.shape[0], inp_dm.shape[0], inp_dm.shape[0]))

        #vxc function for gradient
        vgf = lambda x: model(x, ao_eval, gw)
        print('Generating dmp, moep, mocoep')
        print(f'inp_dm, eri, mo_occ, hc, s shapes: {inp_dm.shape}, {eri.shape}, {mo_occ.shape}, {hc.shape}, {s.shape}')
        dmp, moep, mocoep = xce.utils.get_dm_moe(inp_dm, eri, vgf, mo_occ, hc, s, ogd)
        print(f'dmp, moep, mocoep shapes: {dmp.shape}, {moep.shape}, {mocoep.shape}')
        
        dmp = xce.utils.pad_array(dmp, inp_dm)
        moep = xce.utils.pad_array(moep, moep,  shape=(dmp.shape[0],))
        # print(f'Getting veff from dmp, eri (shapes = {dmp.shape}, {eri.shape})')
        # veff = xce.utils.get_veff()(dmp, eri)
        # print('Getting exc, vxc from dmp')
        # exc, vxc = eqx.filter_value_and_grad(model)(dmp, ao_eval, gw)
        # e_pred = xce.utils.energy_tot()(dmp, hc, veff+vxc+vn)
        # print('e_pred, refE: ', e_pred, refE)
        # rho_pred = xce.utils.get_rho()(dmp, ao_eval)
        # rho_ref = xce.utils.get_rho()(refDM, ao_eval)
        print('Getting gap_pred')
        gap_pred = moep[homo_i+1]-moep[homo_i]
        print('gap_pred, refGap: ', gap_pred, refGap)

        # eL = (e_pred-refE)**2
        eL = 0

        dm_L = jnp.sum( (dmp-refDM)**2)

        gap_L = (gap_pred-refGap)**2

        # return jnp.sqrt(eL+dm_L+gap_L)[0]
        return jnp.sqrt(dm_L+gap_L)

class DM_Gap_loss(eqx.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, model, ao, t, hc, eri, s, mf, gc, gw, inp_dm, mo_occ, ogd, refE, refDM, refGap):

        homo_i = jnp.max(jnp.nonzero(mo_occ, size=inp_dm.shape[0])[0])
        #vxc function for gradient
        vgf = lambda x: model(x, ao, gw)
        print('Generating dmp, moep, mocoep')
        print(f'inp_dm, eri, mo_occ, hc, s shapes: {inp_dm.shape}, {eri.shape}, {mo_occ.shape}, {hc.shape}, {s.shape}')
        dmp, moep, mocoep = xce.utils.get_dm_moe(inp_dm, eri, vgf, mo_occ, hc, s, ogd)
        print(f'dmp, moep, mocoep shapes: {dmp.shape}, {moep.shape}, {mocoep.shape}')
        
        dmp = xce.utils.pad_array(dmp, inp_dm)
        moep = xce.utils.pad_array(moep, moep,  shape=(dmp.shape[0],))
        print('Getting gap_pred')
        gap_pred = moep[homo_i+1]-moep[homo_i]
        print('gap_pred, refGap: ', gap_pred, refGap)

        dm_L = jnp.sum( (dmp-refDM)**2)

        gap_L = (gap_pred-refGap)**2

        return jnp.sqrt(dm_L+gap_L)

In [8]:
gpus = jax.devices(backend='gpu')
cpus = jax.devices(backend='cpu')

In [9]:
gpus, cpus

([cuda(id=0)], [CpuDevice(id=0)])

In [10]:
gws[0].shape

(14429,)

In [11]:
#update docs, only input =2 ??? for MGGA? holdover from sebastian for some reason
xnet = eX(n_input = 2, n_hidden=16, depth=3, use = [1, 2], ueg_limit=True, lob=1.174)
# I guess use default LOB
cnet = eC(n_input = 4, n_hidden=16, depth=3, use = [2, 3], ueg_limit=True)
blankxc = xce.xc.eXC(grid_models = [xnet, cnet], level=3)
p = '/home/awills/Documents/Research/xcquinox/models/pretrained/scan'
# xc = eqx.tree_deserialise_leaves(os.path.join(p, 'xc.eqx'), blankxc)
xc = blankxc
# nlxnet = eX(n_input = 15, use = [], ueg_limit=True, lob=1.174)
# nlcnet = eC(n_input = 13, use = [], ueg_limit=True)

# nlxc = xce.xc.eXC(grid_models = [nlxnet, nlcnet], level=4)

Finished tracing + transforming convert_element_type for pjit in 0.00020575523376464844 sec
Compiling convert_element_type for with global shapes and types [ShapedArray(int64[])]. Argument mapping: (GSPMDSharding({replicated}),).
Finished jaxpr to MLIR module conversion jit(convert_element_type) in 0.00102996826171875 sec
Finished XLA compilation of jit(convert_element_type) in 0.0024569034576416016 sec
Finished tracing + transforming <lambda> for pjit in 0.0004584789276123047 sec
Finished tracing + transforming _threefry_seed for pjit in 0.001994609832763672 sec
Compiling _threefry_seed for with global shapes and types [ShapedArray(int64[])]. Argument mapping: (GSPMDSharding({replicated}),).
Finished jaxpr to MLIR module conversion jit(_threefry_seed) in 0.0018782615661621094 sec
Finished XLA compilation of jit(_threefry_seed) in 0.011458635330200195 sec
Finished tracing + transforming ravel for pjit in 0.00014257431030273438 sec
Finished tracing + transforming threefry_2x32 for pjit 

In [ ]:
xct = xce.train.xcTrainer(model=xc, optim=optax.adamw(1e-2), steps=100, loss = DM_Gap_loss(), do_jit=True)
with jax.default_device(cpus[0]):
    # newm = xct(len(mols), xct.model, mols, mfs, gcs, gws, init_dms, rmooccs, ogds, reftotes, rdms, refgaps)
    newm = xct(len(mols), xct.model, aos, ts, hcs, eris, ss, mfs, gcs, gws, init_dms, rmooccs, ogds, reftotes, rdms, refgaps)

Finished tracing + transforming convert_element_type for pjit in 0.00019979476928710938 sec
Compiling convert_element_type for with global shapes and types [ShapedArray(float64[])]. Argument mapping: (GSPMDSharding({replicated}),).
Finished jaxpr to MLIR module conversion jit(convert_element_type) in 0.00098419189453125 sec
Finished XLA compilation of jit(convert_element_type) in 0.002337217330932617 sec
Finished tracing + transforming broadcast_in_dim for pjit in 0.00019669532775878906 sec
Compiling broadcast_in_dim for with global shapes and types [ShapedArray(float64[])]. Argument mapping: (GSPMDSharding({replicated}),).
Finished jaxpr to MLIR module conversion jit(broadcast_in_dim) in 0.0010530948638916016 sec
Finished XLA compilation of jit(broadcast_in_dim) in 0.010089635848999023 sec
Finished tracing + transforming broadcast_in_dim for pjit in 0.0003597736358642578 sec
Compiling broadcast_in_dim for with global shapes and types [ShapedArray(float64[])]. Argument mapping: (GSPMDS

Epoch 0
Epoch 0 :: Batch 0/8
Generating dmp, moep, mocoep
inp_dm, eri, mo_occ, hc, s shapes: (16, 16), (16, 16, 16, 16), (16,), (16, 16), (16, 16)
spin_scaling = True; input descr to exc shape: (2, 14429, 3)
eX.__call__, rho shape: (2, 14429, 3)


Finished tracing + transforming <lambda> for pjit in 0.00035381317138671875 sec
Finished tracing + transforming matmul for pjit in 0.0003936290740966797 sec
Finished tracing + transforming logaddexp for pjit in 0.0006296634674072266 sec
Finished tracing + transforming softplus for pjit in 0.0018270015716552734 sec
Finished tracing + transforming logaddexp for pjit in 0.0004718303680419922 sec
Finished tracing + transforming real for pjit in 0.00018715858459472656 sec
Finished tracing + transforming exp for pjit in 0.0001976490020751953 sec
Finished tracing + transforming _squeeze for pjit in 0.0001723766326904297 sec
Finished tracing + transforming tanh for pjit in 0.00031876564025878906 sec
Finished tracing + transforming fn for pjit in 0.000457763671875 sec
Finished tracing + transforming <lambda> for pjit in 0.00023317337036132812 sec
Finished tracing + transforming sigmoid for pjit in 0.0001277923583984375 sec
Finished tracing + transforming <lambda> for pjit in 0.00025773048400878

spin_scaling = False; input descr to exc shape: (14429, 4)
eC.__call__, rho shape: (14429, 4)


Finished tracing + transforming fn for pjit in 0.00028228759765625 sec
Finished tracing + transforming _broadcast_arrays for pjit in 0.00011730194091796875 sec
Finished tracing + transforming _where for pjit in 0.0008494853973388672 sec
Finished tracing + transforming _reduce_sum for pjit in 0.0002846717834472656 sec
Finished tracing + transforming <lambda> for pjit in 0.0002601146697998047 sec
Finished tracing + transforming _squeeze for pjit in 0.00010371208190917969 sec
Finished tracing + transforming <lambda> for pjit in 0.00021982192993164062 sec
Finished tracing + transforming <lambda> for pjit in 0.0002067089080810547 sec
Finished tracing + transforming fn for pjit in 0.00020766258239746094 sec
Finished tracing + transforming fn for pjit in 0.0001888275146484375 sec
Finished tracing + transforming _uniform for pjit in 0.0037522315979003906 sec
Finished tracing + transforming _einsum for pjit in 0.0005245208740234375 sec
Finished tracing + transforming swapaxes for pjit in 0.0001

...

Finished tracing + transforming true_divide for pjit in 0.0002675056457519531 sec
Finished tracing + transforming exp for pjit in 0.00015401840209960938 sec
Finished tracing + transforming <lambda> for pjit in 0.00030922889709472656 sec
Finished tracing + transforming fn for pjit in 0.00023865699768066406 sec
Finished tracing + transforming log for pjit in 0.00015926361083984375 sec
Finished tracing + transforming fn for pjit in 0.00027871131896972656 sec
Finished tracing + transforming negative for pjit in 0.00021910667419433594 sec
Finished tracing + transforming true_divide for pjit in 0.0002613067626953125 sec
Finished tracing + transforming exp for pjit in 0.00017189979553222656 sec
Finished tracing + transforming <lambda> for pjit in 0.00023984909057617188 sec
Finished tracing + transforming fn for pjit in 0.0002238750457763672 sec
Finished tracing + transforming log for pjit in 0.0001583099365234375 sec
Finished tracing + transforming fn for pjit in 0.00019598007202148438 sec
Fi

spin_scaling = True; input descr to exc shape: (2, 14429, 3)
eX.__call__, rho shape: (2, 14429, 3)
spin_scaling = False; input descr to exc shape: (14429, 4)
eC.__call__, rho shape: (14429, 4)


Finished tracing + transforming matmul for pjit in 0.00037217140197753906 sec
Finished tracing + transforming fn for pjit in 0.0002696514129638672 sec
Finished tracing + transforming _broadcast_arrays for pjit in 0.0001239776611328125 sec
Finished tracing + transforming _where for pjit in 0.0007562637329101562 sec
Finished tracing + transforming _reduce_sum for pjit in 0.00026297569274902344 sec
Finished tracing + transforming <lambda> for pjit in 0.0002384185791015625 sec
Finished tracing + transforming _squeeze for pjit in 0.00010800361633300781 sec
Finished tracing + transforming <lambda> for pjit in 0.0002694129943847656 sec
Finished tracing + transforming <lambda> for pjit in 0.00019884109497070312 sec
Finished tracing + transforming fn for pjit in 0.00020074844360351562 sec
Finished tracing + transforming fn for pjit in 0.000217437744140625 sec
Finished tracing + transforming _uniform for pjit in 0.003271341323852539 sec
Finished tracing + transforming _einsum for pjit in 0.00050

[16] (8,)
(16, 16) (8, 8)
Spin unpolarized make_rdm1()
dmp, moep, mocoep shapes: (16, 16), (16,), (16, 16)
(16, 16) (16, 16)
(16,) (16,)
Getting gap_pred
gap_pred, refGap:  Traced<ShapedArray(float64[])>with<JVPTrace(level=3/0)> with
  primal = Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
  tangent = Traced<ShapedArray(float64[])>with<JaxprTrace(level=2/0)> with
    pval = (ShapedArray(float64[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f620c5313e0>, in_tracers=(Traced<ShapedArray(float64[]):JaxprTrace(level=2/0)>, Traced<ShapedArray(float64[]):JaxprTrace(level=2/0)>), out_tracer_refs=[<weakref at 0x7f620bab9cb0; to 'JaxprTracer' at 0x7f620babb6a0>], out_avals=[ShapedArray(float64[])], primitive=pjit, params={'jaxpr': { lambda ; a:f64[] b:f64[]. let c:f64[] = sub a b in (c,) }, 'in_shardings': (UnspecifiedValue, UnspecifiedValue), 'out_shardings': (UnspecifiedValue,), 'resource_env': None, 'donated_invars': (False, False), 'name': '<lambda>', '

Compiling make_step for with global shapes and types [ShapedArray(float64[16,2]), ShapedArray(float64[16]), ShapedArray(float64[16,16]), ShapedArray(float64[16]), ShapedArray(float64[16,16]), ShapedArray(float64[16]), ShapedArray(float64[1,16]), ShapedArray(float64[1]), ShapedArray(float64[]), ShapedArray(float64[16,4]), ShapedArray(float64[16]), ShapedArray(float64[16,16]), ShapedArray(float64[16]), ShapedArray(float64[16,16]), ShapedArray(float64[16]), ShapedArray(float64[1,16]), ShapedArray(float64[1]), ShapedArray(int32[]), ShapedArray(float64[16,2]), ShapedArray(float64[16]), ShapedArray(float64[16,16]), ShapedArray(float64[16]), ShapedArray(float64[16,16]), ShapedArray(float64[16]), ShapedArray(float64[1,16]), ShapedArray(float64[1]), ShapedArray(float64[]), ShapedArray(float64[16,4]), ShapedArray(float64[16]), ShapedArray(float64[16,16]), ShapedArray(float64[16]), ShapedArray(float64[16,16]), ShapedArray(float64[16]), ShapedArray(float64[1,16]), ShapedArray(float64[1]), ShapedAr

In [ ]:
borlido = pd.read_excel('/home/awills/Documents/Research/datasets/borlido2019/gg_direct_gaps.xlsx')

In [ ]:
mol.nelectron

In [ ]:
borlido

In [ ]:
with open('/home/awills/Documents/Research/datasets/borlido2019/ggdirect.pkl', 'rb') as f:
    bstruc_list = pickle.load(f)

In [ ]:
smalls = [i for i in bstruc_list if len(i[-1].structure.as_dict()['sites']) <= 4]

In [ ]:
smalls[0]

In [ ]:
smallids = [i[1] for i in smalls]

In [ ]:
smalldf = borlido[borlido['MP_ID'].isin(smallids)]

In [ ]:
traj = []
for small in smalls:
    datidx, mpid, bstruc = small
    at = AseAtomsAdaptor.get_atoms(bstruc.structure)
    at.info['borlido_idx'] = datidx
    at.info['mp_id'] = mpid
    at.info['reference'] = smalldf[smalldf['MP_ID']==mpid]['Experimental'].values[0]
    traj.append(at)

In [ ]:
traj[2].info

In [ ]:
tt = traj[2]

In [ ]:
from pyscf.pbc.tools import pyscf_ase

In [ ]:
pyscf_ase.ase_atoms_to_pyscf(tt)

In [ ]:
write('/home/awills/Documents/Research/datasets/borlido2019/smalls.traj', traj)

In [ ]:
test = smalls[0][-1]

In [ ]:
gtop

In [ ]:
test.as_dict()['lattice_rec']

In [ ]:
at  = AseAtomsAdaptor.get_atoms(test.structure)

In [ ]:
mol.pbc

In [ ]:
gtop.Cell(a=np.array(at.cell), charge=0).spin

In [ ]:
from pyscf.pbc import cc

In [ ]:
cc.

In [ ]:
image = np.array([
[[1, 2, 3], [4, 5, 6], [-1, 8, 9]],
[[10, 11, 12], [13, 14, 15], [16, 17, 18]],
[[19, 20, 21], [22, 23, 24], [25, 26, 27]]
],
dtype=np.float32)
mask = np.array(
[[1, 4, 7],
[10, 4, 16],
[19, 22, 255]],
dtype=np.uint8)

In [ ]:
with MPRester(api_key = '') as mpr:
    docs = mpr.materials.summary.search(
        band_gap=(0, 5), is_stable=True, fields=["material_id"], is_gap_direct = True, num_elements=(0, 4), num_sites=(0, 4),
        has_props=[HasProps.bandstructure]
    )
    gamma_directs = [mpr.get_bandstructure_by_material_id(str(i.material_id)) for i in docs]

In [ ]:
t = gamma_directs[10]

In [ ]:
t.get_band_gap()

In [ ]:
gamgam = [i for i in gamma_directs if i.get_band_gap()['transition'] == '\\Gamma-\\Gamma']

In [ ]:
len(borlido_gg_direct[0][-1].structure.as_dict()['sites'])

In [ ]:
smalls = [(i[1], i[-1]) for i in borlido_gg_direct if len(i[-1].structure.as_dict()['sites']) < 4]

In [ ]:
[i[1].structure.as_dict()['sites'][0]['species'] for i in smalls]

In [ ]:
mpr = MPRester(api_key = '')

In [ ]:
# mp = mpr.get_bandstructure_by_material_id('mp-1266')
mp = smalls[0]
mps = mp.structure.as_dict()
mp.get_band_gap()

In [ ]:
rets = mps
at_coor_xyz = [ (i['species'][0]['element'], i['xyz']) for i in rets['sites']]
at_coor_abc = [ (i['species'][0]['element'], [rets['lattice']['a']*j for j in i['abc']]) for i in rets['sites']]
lat = np.array(rets['lattice']['matrix'])
if np.linalg.det(lat) < 1:
    print('left handed array, switching')
    lat = -np.array([lat[-1], lat[1], lat[0]])

In [ ]:
np.linalg.det(lat)

In [ ]:
traj

In [ ]:
at_coor_abc

In [ ]:
cell = gtop.Cell(verbose=9)
cell.atom = at_coor_abc
cell.a = lat
cell.basis = 'def2-svp'
cell.pseudo = 'gth-pbe'
cell.exp_to_discard = 0.1
cell.build()
# kpts = cell.make_kpts([2,2,2])
mf = dftp.RKS(cell, xc='pbe')
# mf = scfp.RHF(cell)
e = mf.kernel()

In [ ]:
mf.mo_occ, mf.mo_energy

In [ ]:
mf.mo_energy - mf.mo_energy[mf.mo_occ == 0][0], mf.mo_energy - mf.mo_energy[mf.mol.nelectron//2-1], mf.mo_energy, mf.mo_occ

In [ ]:
a1 = bulk('Si', a=3.867114, b=3.867114, c=3.867114, alpha=60)

In [ ]:
a1.cell

In [ ]:
gtop.Cell?

In [ ]:
gtop

In [ ]:
mfs = []
mols = []
energies = []
dms = []
ao_evals = []
gws = []
eris = []
mo_occs = []
hcs = []
vs = []
ts = []
ss = []
hologaps = []
ogds = []

cell = gtop.Cell()
a = 5.43
cell.atom = [['Si', [0,0,0]],
              ['Si', [a/4,a/4,a/4]]]
cell.a = jnp.asarray([[0, a/2, a/2],
                     [a/2, 0, a/2],
                     [a/2, a/2, 0]])
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.exp_to_discard = 0.1
cell.build()
kpts = cell.make_kpts([2,2,2])
mf = dftp.RKS(cell, xc='pbe0')
mf2 = dftp.KRKS(cell, xc='pbe0', kpts=kpts)
e = mf.kernel()
# e2 = mf2.kernel()

In [ ]:
mf

In [ ]:
mfs = []
mols = []
energies = []
dms = []
ao_evals = []
gws = []
eris = []
mo_occs = []
hcs = []
vs = []
ts = []
ss = []
hologaps = []
ogds = []

mfs.append(mf)
dm = mf.make_rdm1()
dmj = jnp.array(dm)
dmj.flags = dm.flags
ao_eval = jnp.array(mf._numint.eval_ao(mf.mol, mf.grids.coords, deriv=2))
energies.append(jnp.array(mf.get_veff().exc))
dms.append(dmj)
ogds.append(dm.shape)
ao_evals.append(jnp.array(ao_eval))
gws.append(jnp.array(mf.grids.weights))
ts.append(jnp.array(mf.mol.intor('int1e_kin')))
vs.append(jnp.array(mf.mol.intor('int1e_nuc')))
mo_occs.append(jnp.array(mf.mo_occ))
hcs.append(jnp.array(mf.get_hcore()))
eris.append(jnp.array(mf.mol.intor('int2e')))
ss.append(jnp.linalg.inv(jnp.linalg.cholesky(mf.mol.intor('int1e_ovlp'))))
hologaps.append(jnp.array(mf.mo_energy[mf.mo_occ == 0][0] - mf.mo_energy[mf.mo_occ > 1][-1]))

In [ ]:
class Band_gap_1shot_loss(eqx.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, model, ao_eval, gw, dm, eri, mo_occ, hc, s, ogd, refgap, mf, alpha0=0.7):
        vgf = lambda x: model(x, ao_eval, gw, mf)
        dmp, moep, mocp = xce.utils.get_dm_moe(dm, eri, vgf, mo_occ, hc, s, ogd, alpha0)
        
        efermi = moep[mf.mol.nelectron//2-1]
        moep -= efermi
        # print(moep)
        moep_gap = jnp.min(moep)
        # print(moep_gap)
        loss = jnp.sqrt( (moep_gap - refgap)**2)
        # print(loss)
        return jnp.sqrt( (moep_gap - refgap)**2)


In [ ]:
xce.net.eX?

In [ ]:
#update docs, only input =2 ??? for MGGA? holdover from sebastian for some reason
xnet = xce.net.eX(n_input = 2, n_hidden=32, depth=4, use = [1, 2], ueg_limit=True, lob=1.174)
# I guess use default LOB
cnet = xce.net.eC(n_input = 4, n_hidden=32, depth=4, use = [2, 3], ueg_limit=True)
blankxc = xce.xc.eXC(grid_models = [xnet, cnet], level=3)
p = '/home/awills/Documents/Research/xcquinox/models/pretrained/scan'
# xc = eqx.tree_deserialise_leaves(os.path.join(p, 'xc.eqx'), blankxc)
xc = blankxc
nlxnet = xce.net.eX(n_input = 15, use = [], ueg_limit=True, lob=1.174)
nlcnet = xce.net.eC(n_input = 13, use = [], ueg_limit=True)

nlxc = xce.xc.eXC(grid_models = [nlxnet, nlcnet], level=4)

In [ ]:

xc(dms[0], ao_evals[0], gws[0])

In [ ]:
class Band_gap_janak_loss(eqx.Module):
    def __init__(self):
        """
        Initializer for the loss module, which attempts to find loss bang gaps w.r.t. reference

        .. todo: Make more robust for non-local descriptors
        """
        super().__init__()

    def __call__(self, model, ao_eval, gw, dm, eri, mo_occ, hc, s, ogd, refgap, mf, alpha0=0.7):
        """
        Forward pass for loss object

        NOTE: This differs from HoLu loss in that it selects the deepest minimum w.r.t. the LUMO (Fermi energy)

        :param model: The model that will be used in generating the molecular orbital energies ('band' energies)
        :type model: xcquinox.xc.eXC
        :param ao_eval: The atomic orbitals evaluated on the grid for the given molecule
        :type ao_eval: jax.Array
        :param gw: The grid weights associated to the current molecule's grids
        :type gw: jax.Array
        :param dm: Input reference density matrix for use during the one-shot forward pass to generate the new DM
        :type dm: jax.Array
        :param eri: Electron repulsion integrals associated with this molecule
        :type eri: jax.Array
        :param mo_occ: The molecule's molecular orbital occupation numbers
        :type mo_occ: jax.Array
        :param hc: The molecule's core Hamiltonian
        :type hc: jax.Array
        :param s: The molecule's overlap matrix
        :type s: jax.Array
        :param ogd: The original dimensions of this molecule's density matrix, used if padded to constrict the eigendecomposition to a relevant shape
        :type ogd: jax.Array
        :param refgap: The reference gap to optimzie against
        :type refgap: jax.Array
        :param mf: A pyscf(ad) converged calculation kernel if self.level > 3, used for building the CIDER nonlocal descriptors, defaults to None
        :type mf: pyscfad.dft.RKS kernel
        :param alpha0: The mixing parameter for the one-shot density matrix generation, defaults to 0.7
        :type alpha0: float, optional
        :return: Root-squared error between predicted gap (minimum of molecular energies) and the reference
        :rtype: jax.Array
        """
        def janak_theorem_deriv(model, ao_eval, gw, dm, eri, hc, s, ogd, alpha0=0.7):
            def ret_func(mo_occ):
                vgf = lambda x: model(x, ao_eval, gw)
                dmp, moep, mocp = xce.utils.get_dm_moe(dm, eri, vgf, mo_occ, hc, s, ogd, alpha0=alpha0)
                return model(dmp, ao_eval, gw)
        
            return ret_func

        janak_f = janak_theorem_deriv(model, ao_eval, gw, dm, eri, hc, s, ogd, alpha0)
        homo_i = jnp.max(jnp.nonzero(mo_occ, size=dm.shape[0])[0])

        e, derivs = eqx.filter_value_and_grad(janak_f)(mo_occ)

        pred_diff = derivs[homo_i+1] - derivs[homo_i]
        
        loss = jnp.sqrt( (pred_diff - refgap)**2)
        # print(loss)
        return loss


In [ ]:
def janak_theorem_deriv(model, ao_eval, gw, dm, eri, moocc, hc, s, ogd, alpha0=0.7):
    def ret_func(mo_occ):
        vgf = lambda x: model(x, ao_eval, gw)
        dmp, moep, mocp = xce.utils.get_dm_moe(dm, eri, vgf, mo_occ, hc, s, ogd, alpha0=alpha0)
        return model(dmp, ao_eval, gw)

    return ret_func
    
    
    

In [ ]:
checkd = janak_theorem_deriv(xc, ao_evals[0], gws[0], dms[0], eris[0], mo_occs[0], hcs[0], ss[0], ogds[0], alpha0=0.7)
eqx.filter_value_and_grad(checkd)(mo_occs[0])

In [ ]:
xct = xce.train.xcTrainer(model=xc, optim=optax.adamw(1e-2), steps=100, loss = Band_gap_janak_loss(), do_jit=True)
newm = xct(1, xct.model, ao_evals, gws, dms, eris, mo_occs, hcs, ss, ogds, [1.17], mfs)

In [ ]:
e1 = nlxc(dms[0], ao_evals[0], gws[0], mfs[0])
e2 = newm(dms[0], ao_evals[0], gws[0], mfs[0])

In [ ]:
e1, e2

In [ ]:
vgf1 = lambda x: xc(x, ao_evals[0], gws[0], mfs[0])
vgf2 = lambda x: newm(x, ao_evals[0], gws[0], mfs[0])
dm1, moe1, moc1 = xce.utils.get_dm_moe(dms[0], eris[0], vgf1, mo_occs[0], hcs[0], ss[0], ogds[0])
dm2, moe2, moc2 = xce.utils.get_dm_moe(dms[0], eris[0], vgf2, mo_occs[0], hcs[0], ss[0], ogds[0])

In [ ]:
print(moe1 - moe1[mf.mol.nelectron//2-1])
print(moe2 - moe2[mf.mol.nelectron//2-1])

In [ ]:
mf.mo_energy

In [ ]:
# vbmax = -99
for en in b1[0]:
    vb_k = en[cell.nelectron//2-1]
    print('This vb_k', vb_k)
    if vb_k > vbmax:
        vbmax = vb_k
e_kn = [en - vbmax for en in b1[0]]

In [ ]:
e_kn

In [ ]:
mf2 = scfp.RHF(cell)
e2 = mf2.kernel()

In [ ]:
cell.nelectron//2-1

In [ ]:
t1 = mf2.mo_energy 
t2 = mf2.mo_energy - mf2.mo_energy[cell.nelectron//2-1]

In [ ]:
t2[jnp.where(abs(t2[jnp.where( (t2 < 0) )[0]]) > 1e-4)[0]]

In [ ]:
dm2 = mf2.make_rdm1()

In [ ]:
dmk = mf.make_rdm1()

In [ ]:
mpr = MPRester(api_key = '')
mpid = 'mp-149'
# ret = mpr.get_bandstructure_by_material_id(mpid)
# rets = ret.structure.as_dict()
ret = mpr.get_structure_by_material_id(mpid, conventional_unit_cell=False)
rets = ret.as_dict()
at_coor_xyz = [ (i['species'][0]['element'], [-j for j in i['xyz']]) for i in rets['sites']]
at_coor_abc = [ (i['species'][0]['element'], [rets['lattice']['a']*j for j in i['abc']]) for i in rets['sites']]
cella = -np.asarray(rets['lattice']['matrix'])

In [ ]:
at_coor_xyz, cella

In [ ]:
cell = gtop.Cell()
cell.atom = at_coor_xyz
cell.a = cella.T
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.exp_to_discard = 0.1
cell.build()
kpts = cell.make_kpts([2,2,2])
mf = scfp.KRHF(cell, kpts=kpts)
e = mf.kernel()

In [ ]:
cella